In [ ]:
from nibabel import load, save, Nifti1Image
from numpy import absolute

analysis = ['tstat1_inv','tstat2_inv','tstat2_inv']

home = '/moochie/user_data/CamachoCat/Aggregate_anats/proc/group_data/GLMs/smooth4_negaff_under11/'

for a in analysis:
    file = home + 'tstat_files/randomise_%s.nii.gz' % a
    temp_nii = load(file)
    temp_data = temp_nii.get_data()
    new_data = absolute(temp_data)
    new_nii = Nifti1Image(new_data,header=temp_nii.header,affine=temp_nii.affine)
    new_file = home + 'cluster_files/clusters_%s.nii.gz' % a
    save(new_nii,new_file)

In [4]:
from nipype.interfaces.fsl import Cluster

analysis = ['tstat1_inv','tstat2_inv','tstat3_inv']

home = '/moochie/user_data/CamachoCat/Aggregate_anats/proc/group_data/GLMs/smooth4_negaff_under11/'

for a in analysis:
    file = home + 'tstat_files/randomise_%s.nii.gz' % a
    cl = Cluster()
    cl.inputs.in_file = file
    cl.inputs.out_localmax_txt_file = home + 'clusters_file/cluster_%s.txt' % a
    cl.inputs.threshold = 3.15
    cl.dlh=4, 
    cl.pthreshold=0.05, 
    cl.volume=1472512,
    cl.inputs.out_index_file = home + 'clusters_file/clusters_%s.nii.gz' % a
    cl.run()

190920-16:00:37,504 interface INFO:
	 stdout 2019-09-20T16:00:37.503893:Cluster Index	Voxels	MAX	MAX X (vox)	MAX Y (vox)	MAX Z (vox)	COG X (vox)	COG Y (vox)	COG Z (vox)
190920-16:00:37,506 interface INFO:
	 stdout 2019-09-20T16:00:37.503893:126	216	5.01	180	141	112	180	140	114
190920-16:00:37,508 interface INFO:
	 stdout 2019-09-20T16:00:37.503893:125	149	4.67	70	112	167	70.9	111	166
190920-16:00:37,509 interface INFO:
	 stdout 2019-09-20T16:00:37.503893:124	142	4.31	171	138	158	174	140	160
190920-16:00:37,510 interface INFO:
	 stdout 2019-09-20T16:00:37.503893:123	107	4.74	109	63	158	109	62.3	160
190920-16:00:37,511 interface INFO:
	 stdout 2019-09-20T16:00:37.503893:122	72	4.13	138	137	104	138	137	104
190920-16:00:37,512 interface INFO:
	 stdout 2019-09-20T16:00:37.503893:121	68	3.82	155	194	131	154	193	132
190920-16:00:37,514 interface INFO:
	 stdout 2019-09-20T16:00:37.503893:120	67	4.13	179	126	121	179	126	121
190920-16:00:37,515 interface INFO:
	 stdout 2019-09-20T16:00:37.503893

In [6]:
def extract_cluster_betas(cluster_index_file, sample_betas, min_clust_size, subject_ids):
    from nibabel import load, save, Nifti1Image
    from pandas import DataFrame, Series
    from numpy import unique, zeros_like, invert
    from nipype.interfaces.fsl.utils import ImageMeants
    from os.path import abspath, basename
    
    subject_ids = sorted(subject_ids)
    sample_data = DataFrame(subject_ids, index=None, columns=['Subject'])
    
    cluster_nifti = load(cluster_index_file)
    cluster_data = cluster_nifti.get_data()
    clusters, cluster_sizes = unique(cluster_data, return_counts=True)
    cluster_sizes = cluster_sizes[clusters>min_clust_size]
    clusters = clusters[clusters>min_clust_size]
    ind_filename = basename(cluster_index_file) 
    out_prefix = ind_filename[:-7]
    
    for clust_idx in clusters:
        temp = zeros_like(cluster_data)
        temp[cluster_data==clust_idx] = 1
        temp_nii = Nifti1Image(temp,cluster_nifti.affine)
        temp_file = 'temp_clust_mask.nii.gz'
        save(temp_nii, temp_file)

        eb = ImageMeants()
        eb.inputs.in_file = sample_betas
        eb.inputs.mask = temp_file
        eb.inputs.out_file = 'values.txt'
        eb.run()
        betas = open('values.txt').read().splitlines()
        sample_data['clust' + str(clust_idx)] = Series(betas, index=sample_data.index)
    
    sample_data.to_csv(out_prefix+'_extracted_values.csv')
    extracted_betas_csv = abspath(out_prefix+'_extracted_values.csv')
    
    print('###### ' + out_prefix + ' #######')
    print('cluster labels: '+str(clusters))
    print('cluster sizes: '+str(cluster_sizes))
    return(extracted_betas_csv)

In [8]:
from glob import glob
from pandas import read_csv
index_files = glob(home + 'clusters_file/clusters_*.nii.gz')
sample_betas = '/moochie/user_data/CamachoCat/Aggregate_anats/workflows/grouplevel_negaffect_under11/select_usable_data/data_merged.nii.gz'

subject_info = read_csv('/moochie/user_data/CamachoCat/Aggregate_anats/proc/group_data/smooth4/featureset_key.csv',index_col=0)
subject_info = subject_info[subject_info['Age_yrs']<11].copy()
usable_subs = subject_info[subject_info['activity_level']>=0]
subject_ids = usable_subs['freesurferID']

min_clust_size = 100

for clusters in index_files:
    extract_cluster_betas(clusters, sample_betas, min_clust_size, subject_ids)

###### clusters_tstat1_inv #######
cluster labels: [ 51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68
  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86
  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104
 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122
 123 124 125 126]
cluster sizes: [  6   6   6   7   7   7   7   7   7   8   8   8   8   8   8   8   8   8
   8   8   9   9  10  10  11  11  11  11  11  11  12  12  13  13  13  13
  16  20  21  21  21  22  22  23  23  24  24  25  25  25  26  27  29  30
  30  30  31  32  36  38  38  40  40  41  44  47  53  59  61  67  68  72
 107 142 149 216]
###### clusters_tstat2_inv #######
cluster labels: [ 51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68
  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86
  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104
 105 106 107 108 109 110 111 112 113 114 115

In [9]:
def get_cluster_peaks(clusters_file, stat_file):
    from nibabel import load, save, Nifti1Image
    from pandas import DataFrame, Series
    from numpy import unique, unravel_index, max
    
    # load up clusters
    clusters_nii = load(clusters_file)
    clusters_data = clusters_nii.get_data()
    cluster_labels, cluster_sizes = unique(clusters_data, return_counts=True)
    cluster_sizes = cluster_sizes[cluster_labels>0]
    cluster_labels = cluster_labels[cluster_labels>0]
    
    # set up dataframe
    cluster_info = DataFrame(columns=['clust_num','peak','num_voxels','X','Y','Z'])
    cluster_info['clust_num'] = Series(cluster_labels,index=None)
    
    for i in range(0,len(cluster_labels)):
        # load up stat image
        stat_nii = load(stat_file)
        stat_data = stat_nii.get_data()
        stat_data[clusters_data!=cluster_labels[i]]=0
        location=unravel_index(stat_data.argmax(), stat_data.shape)
        cluster_info.iloc[i,0]=cluster_labels[i]
        cluster_info.iloc[i,1]=max(stat_data)
        cluster_info.iloc[i,2]=cluster_sizes[i]
        cluster_info.iloc[i,3]=location[0]
        cluster_info.iloc[i,4]=location[1]
        cluster_info.iloc[i,5]=location[2]
    
    out_prefix = clusters_file[:-7]
    cluster_info.to_csv(out_prefix + '_peaks.csv')
    return(cluster_info)

In [10]:
index_files = glob(home + 'clusters_file/clusters_*.nii.gz')
index_files = sorted(index_files)

stat_files=glob(home + 'tstat_files/randomise_*.nii.gz')
stat_files = sorted(stat_files)

for j in range(0,len(stat_files)):
    get_cluster_peaks(index_files[j], stat_files[j])

IndexError: list index out of range

In [ ]:
from pandas import read_csv, DataFrame, merge
study_home = '/moochie/Cat/Aggregate_anats/GMD_ML'
sub_data_file = study_home + '/doc/subjectinfo.csv'

subject_info = read_csv(sub_data_file, index_col=None)
cluster_info = read_csv('/moochie/Cat/Aggregate_anats/GMD_ML/ml_trainingset/clusters_irritability_extracted_values.csv',index_col=0)

new = subject_info.merge(cluster_info, left_on='Subject', right_on='Subject')
new.columns


In [ ]:
from pandas import read_csv
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='poster',style='white')

data = read_csv('/moochie/Cat/Aggregate_anats/GMD_ML/ml_trainingset/clusters/GMDforfigures.csv',index_col=0)

for reg in ['Loc_clust465','Rstg_clust447','Rthal_clust472','Rfusi_clust450','Lpcg_clust468']:
    sns.lmplot(x='age',y=reg,data=data, ci=None, lowess=True,scatter=False,line_kws={'color':'black'})
    plt.savefig('/moochie/Cat/Aggregate_anats/GMD_ML/ml_trainingset/clusters/age_'+reg+'_lowess.svg')
    
for reg in ['Rmsfg_clust885','Lmfg_clust940','Ripl_clust1061','Rips_clust1110','Rpcg_clust1128']:
    sns.lmplot(x='Temper_Loss',y=reg,data=data, ci=None, lowess=True,scatter=False,line_kws={'color':'black'})
    plt.savefig('/moochie/Cat/Aggregate_anats/GMD_ML/ml_trainingset/clusters/TL_'+reg+'_lowess.svg')
    
for reg in ['mSFG_clust756','Rsma_clust766','Lmfg_clust841','Rmofc_clust859','Lipl_clust866']:
    sns.lmplot(x='General_Aggression',y=reg,data=data, ci=None, lowess=True,scatter=False,line_kws={'color':'black'})
    plt.savefig('/moochie/Cat/Aggregate_anats/GMD_ML/ml_trainingset/clusters/GA_'+reg+'_lowess.svg')